In [1]:
!pip install boto3 pyarrow pandas

In [2]:
import boto3

ACCESS_KEY = "71e05046d84a44308f53c118b502cb6a"
SECRET_KEY = "da42757432504c92b9f2d21e5c6e5634"

s3 = boto3.client(
    service_name="s3",
    region_name="kr-central-2",
    endpoint_url="https://objectstorage.kr-central-2.kakaocloud.com",
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
)


In [7]:
import io
import pyarrow.parquet as pq
import pandas as pd

bucket_name = "data-catalog-bucket"
key = (
    "data-catalog-dir/"
    "user_behavior_prediction_batch/"
    "part-00199-cf023109-0dc2-4dfa-bb0d-6e21d5dc7dd7-c000.snappy.parquet"
)

obj = s3.get_object(Bucket=bucket_name, Key=key)
buf = io.BytesIO(obj["Body"].read())
table = pq.read_table(buf)
df = table.to_pandas()

df.head(10)


,session_id,user_id,gender,age,current_state,search_count,cart_item_count,page_depth,last_action_elapsed,next_state,timestamp,avg_response_time,current_product_id,session_duration,query_params,search_keyword,product_id
0,02ddfbcc-40ab-441f-a697-acefbfdb05c1,None,None,NaN,/add_user,0,0,1,0.0,/search,2025-07-30 12:55:25,0.020,NaN,0.0,None,None,None
1,02ddfbcc-40ab-441f-a697-acefbfdb05c1,None,None,NaN,/login,3,0,6,2.0,None,2025-07-30 12:55:34,0.020,NaN,9.0,None,None,None
2,02ddfbcc-40ab-441f-a697-acefbfdb05c1,None,None,NaN,/product,2,0,4,1.0,/search,2025-07-30 12:55:28,0.020,116.0,3.0,id=116,None,116
3,02ddfbcc-40ab-441f-a697-acefbfdb05c1,None,None,NaN,/search,1,0,2,1.0,/search,2025-07-30 12:55:26,0.020,NaN,1.0,None,Sunglasses,None
4,02ddfbcc-40ab-441f-a697-acefbfdb05c1,None,None,NaN,/search,2,0,3,1.0,/product,2025-07-30 12:55:27,0.020,NaN,2.0,None,iphon,None
5,02ddfbcc-40ab-441f-a697-acefbfdb05c1,None,None,NaN,/search,3,0,5,4.0,/login,2025-07-30 12:55:32,0.020,NaN,7.0,None,cofee,None
6,0579f6b8-cc11-4e47-9902-9deec78aa5db,None,None,NaN,/,0,0,2,1.0,/search,2025-07-30 20:39:41,0.016,NaN,1.0,None,None,None
7,0579f6b8-cc11-4e47-9902-9deec78aa5db,None,None,NaN,/add_user,0,0,1,0.0,/,2025-07-30 20:39:40,0.016,NaN,0.0,None,None,None
8,0579f6b8-cc11-4e47-9902-9deec78aa5db,None,None,NaN,/login,1,0,4,2.0,/cart/add,2025-07-30 20:39:45,0.016,NaN,5.0,None,None,None
9,0579f6b8-cc11-4e47-9902-9deec78aa5db,None,None,NaN,/search,1,0,3,2.0,/login,2025-07-30 20:39:43,0.016,NaN,3.0,None,Bluetooth,None
